# Data Preprocessing for MIMIC-III Demo

> **Overview:**  
> This notebook handles the initial loading and preprocessing of the MIMIC-III Clinical Database Demo files.  
> We will:
> - Load core CSV tables (`PATIENTS.csv`, `ADMISSIONS.csv`, `NOTEEVENTS.csv`, `ICD9_DIAGNOSES.csv`)
> - Merge and clean data
> - Perform basic feature engineering
> - Save a processed dataset for downstream bias-analysis tasks  
>
> _Adapted from the “Hurtful Words” repository’s `get_data.py`, simplified for the demo CSVs._

---


In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import drive # If your CSVs are on Google Drive

# Mount Google Drive (if your data is there)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **Block 2: Load MIMIC-III Demo CSV Files**

> In this block, we load the core data tables from the MIMIC-III Demo dataset. The demo is provided as CSV files.
> **Make sure to upload these CSVs to your Colab environment or mount them from Google Drive, then adjust the file paths below accordingly.**

**Common demo filenames:**

* `PATIENTS.csv`
* `ADMISSIONS.csv`
* `NOTEEVENTS.csv`
* `ICD9_DIAGNOSES.csv` (often included and useful)

---


In [8]:
# --- Adjust these paths to where your CSV files are located ---
# If in Google Drive, it might be '/content/drive/MyDrive/path_to_your_csvs/'
BASE_DATA_PATH = "/content/drive/MyDrive/CS598 Final Project/FULL MIMIC-III CSV's"
# Common MIMIC-III Demo CSV filenames (verify and adjust if yours are different)
patients_csv = os.path.join(BASE_DATA_PATH, "PATIENTS.csv")
admissions_csv = os.path.join(BASE_DATA_PATH, "ADMISSIONS.csv")
notes_csv = os.path.join(BASE_DATA_PATH, "NOTEEVENTS.csv")
diagnoses_csv = os.path.join(BASE_DATA_PATH, "DIAGNOSES_ICD.csv") # Often named DIAGNOSES_ICD.csv

try:
    patients_df = pd.read_csv(patients_csv)
    admissions_df = pd.read_csv(admissions_csv)
    notes_df = pd.read_csv(notes_csv)
    diagnoses_df = pd.read_csv(diagnoses_csv)

    print("Successfully loaded PATIENTS.csv, ADMISSIONS.csv, NOTEEVENTS.csv, and DIAGNOSES_ICD.csv")
    print("\nPatients Data Sample:")
    print(patients_df.head(2))
    print("\nAdmissions Data Sample:")
    print(admissions_df.head(2))
    print("\nNoteEvents Data Sample:")
    print(notes_df.head(2))
    print("\nDiagnoses ICD Data Sample:")
    print(diagnoses_df.head(2))

except FileNotFoundError as e:
    print(f"Error loading CSV files: {e}")
    print("Please ensure your CSV files are correctly named and paths are set.")
    # Set to None if loading fails to prevent errors in subsequent cells
    patients_df, admissions_df, notes_df, diagnoses_df = None, None, None, None

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.


## **Block 3: Initial Data Merging**

> In this step, we consolidate the demo tables into one DataFrame:
>
> 1. Merge patient demographics with admission details
> 2. Append clinical notes (`NOTEEVENTS.csv`)
> 3. Aggregate ICD-9 codes per admission and merge them in

> **Tip:**
> Verify that your key columns (`SUBJECT_ID`, `HADM_ID`) align across DataFrames before merging.

In [ ]:
# Block 3: Initial Data Merging
if patients_df is not None and admissions_df is not None and notes_df is not None and diagnoses_df is not None:
    # Standardize column names to lowercase early on for consistency
    patients_df.columns = patients_df.columns.str.lower()
    admissions_df.columns = admissions_df.columns.str.lower()
    notes_df.columns = notes_df.columns.str.lower()
    diagnoses_df.columns = diagnoses_df.columns.str.lower()

    # Merge patients and admissions
    # When 'row_id' is in both, pandas creates 'row_id_x' (from left, patients_df)
    # and 'row_id_y' (from right, admissions_df)
    df = pd.merge(patients_df, admissions_df, on='subject_id', how='inner')
    print(f"Shape after merging patients and admissions: {df.shape}")
    print(f"Columns in df after pat-adm merge: {df.columns.tolist()}")


    # Ensure HADM_ID types are consistent for merging
    # For notes_df
    if 'hadm_id' in notes_df.columns:
        notes_df.dropna(subset=['hadm_id'], inplace=True) # Drop rows where hadm_id is NaN before astype
        if not notes_df.empty: # only astype if not empty
             notes_df['hadm_id'] = notes_df['hadm_id'].astype('int64')

    # For the main df (derived from patients and admissions)
    if 'hadm_id' in df.columns:
         df.dropna(subset=['hadm_id'], inplace=True) # Drop rows where hadm_id is NaN
         if not df.empty: # only astype if not empty
            df['hadm_id'] = df['hadm_id'].astype('int64')


    # Filter notes_df (it's known to be empty in your demo, so notes_df_filtered will be empty)
    notes_df_filtered = notes_df.dropna(subset=['hadm_id'])
    print(f"Shape of notes_df_filtered: {notes_df_filtered.shape}")

    # Define columns to conceptually merge from notes.
    # These will be added as NaN columns since notes_df_filtered is empty.
    note_cols_to_merge = ['row_id', 'chartdate', 'charttime', 'category', 'text'] # subject_id and hadm_id are merge keys

    # Check which of these columns actually exist in the (empty) notes_df_filtered header
    # This ensures robustness if the demo's empty NOTEEVENTS.csv has a slightly different schema.
    note_cols_present_in_header = [col for col in note_cols_to_merge if col in notes_df_filtered.columns]

    if not notes_df_filtered.empty: # This condition will be false for your demo
        # If notes were present, merge them and add a suffix to distinguish note columns
        # Ensure merge keys for notes_df_filtered also have suffix if 'subject_id' or 'hadm_id' were also in note_cols_to_merge
        # and also suffixed. For simplicity, assuming merge keys are not in note_cols_to_merge directly for suffixing.
        df_notes_to_merge_selected = notes_df_filtered[['subject_id', 'hadm_id'] + note_cols_present_in_header]
        df = pd.merge(df,
                      df_notes_to_merge_selected.add_suffix('_note'),
                      left_on=['subject_id', 'hadm_id'],
                      right_on=['subject_id_note', 'hadm_id_note'], # Match suffixed keys from right
                      how='left')
        # Drop redundant suffixed merge keys from the right table if they were added
        if 'subject_id_note' in df.columns and 'subject_id' in df.columns : df.drop(columns=['subject_id_note'], inplace=True)
        if 'hadm_id_note' in df.columns and 'hadm_id' in df.columns : df.drop(columns=['hadm_id_note'], inplace=True)

    else:
        print("NoteEvents data is empty. Columns from notes (e.g., TEXT, CATEGORY) will be all NaN.")
        # Add the expected note columns as NaN columns to df to maintain structure
        for col_name in note_cols_present_in_header:
            df[col_name + '_note'] = np.nan

    print(f"Shape after merging with notes: {df.shape}")
    print(f"Columns in df after notes merge: {df.columns.tolist()}")


    # Aggregate ICD-9 codes per admission
    icd_col_name = 'icd9_code' # This is the column name in your DIAGNOSES_ICD.csv
    if icd_col_name not in diagnoses_df.columns:
        print(f"Warning: Column '{icd_col_name}' not found in diagnoses_df. Skipping ICD merge.")
    else:
        # Ensure hadm_id in diagnoses_df is also of a compatible type for merging
        diagnoses_df.dropna(subset=['hadm_id'], inplace=True)
        if not diagnoses_df.empty:
            diagnoses_df['hadm_id'] = diagnoses_df['hadm_id'].astype('int64')

            diagnoses_grouped = diagnoses_df.groupby('hadm_id')[icd_col_name].apply(list).reset_index()
            diagnoses_grouped.rename(columns={icd_col_name: 'icd9_codes_list'}, inplace=True)

            df = pd.merge(df, diagnoses_grouped, on='hadm_id', how='left')
            print(f"Shape after merging with diagnoses: {df.shape}")

            print("\nMerged DataFrame sample with ICD codes:")
            # Construct list of columns to print, including a row_id if available
            cols_to_print = ['subject_id', 'hadm_id']
            if 'row_id_y' in df.columns: # row_id from admissions_df
                cols_to_print.append('row_id_y')
            elif 'row_id_x' in df.columns: # row_id from patients_df
                cols_to_print.append('row_id_x')
            # No simple 'row_id' will exist after the first merge due to suffixes

            cols_to_print.append('ethnicity') # This is now lowercase

            if 'icd9_codes_list' in df.columns:
                cols_to_print.append('icd9_codes_list')

            # Ensure all selected columns for printing actually exist in df
            final_cols_to_print = [col for col in cols_to_print if col in df.columns]
            print(df[final_cols_to_print].head(3))
        else:
            print("Diagnoses DataFrame became empty after dropping NaNs in hadm_id. Skipping ICD merge.")

else:
    print("One or more DataFrames (patients_df, admissions_df, notes_df, diagnoses_df) not loaded. Skipping merging.")
    df = None # Ensure df is None if prerequisites aren't met

Shape after merging patients and admissions: (58976, 26)
Columns in df after pat-adm merge: ['row_id_x', 'subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'row_id_y', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag', 'has_chartevents_data']
Shape of notes_df_filtered: (1851344, 11)
Shape after merging with notes: (1851959, 31)
Columns in df after notes merge: ['row_id_x', 'subject_id', 'gender', 'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'row_id_y', 'hadm_id', 'admittime', 'dischtime', 'deathtime', 'admission_type', 'admission_location', 'discharge_location', 'insurance', 'language', 'religion', 'marital_status', 'ethnicity', 'edregtime', 'edouttime', 'diagnosis', 'hospital_expire_flag', 'has_chartevents_data', 'row_id_note', 'chartdate_note', 'charttime_note', 

## **Block 4: Data Cleaning & Feature Engineering (Simplified)**

> In this section, we:
>
> * Convert relevant date columns to datetime objects
> * Estimate patient age at the time of each note (subject to demo data reliability)
> * Clean and map `ETHNICITY` and `LANGUAGE` fields using a simplified version of the original mappings
>
> *Note:* Demo dates are often shifted and may not align perfectly with `CHARTDATE`. Treat age calculations as approximate.

---

In [ ]:
# Block 4: Data Cleaning and Feature Engineering (Further Revised for Robust Age Calculation)

if df is not None:
    # Convert date columns (lowercase)
    date_cols = ['dob', 'dod', 'admittime', 'dischtime', 'deathtime',
                 'chartdate_note', 'dod_hosp', 'dod_ssn', 'edregtime', 'edouttime']
    for col in date_cols:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    # Initialize age columns
    df['age_at_note'] = np.nan
    df['age_at_admission'] = np.nan

    # --- Age Calculation ---
    # Attempt for age_at_note (will likely be NaN due to empty notes)
    if 'dob' in df.columns and 'chartdate_note' in df.columns:
        note_date_valid_mask = (
            df['dob'].notna() &
            df['chartdate_note'].notna() &
            (df['chartdate_note'] > df['dob']) &
            (df['dob'].dt.year > 1850) # Filter out extremely old DOBs
        )
        if note_date_valid_mask.any():
            try:
                time_diff_note = df.loc[note_date_valid_mask, 'chartdate_note'] - df.loc[note_date_valid_mask, 'dob']
                df.loc[note_date_valid_mask, 'age_at_note'] = time_diff_note.dt.days / 365.25
                df.loc[df['age_at_note'] > 89, 'age_at_note'] = 90.0 # Age capping
                print("Calculated 'age_at_note' where possible.")
            except OverflowError:
                print("OverflowError encountered during 'age_at_note' calculation for some rows. These will remain NaN.")
        else:
            print("'chartdate_note' is likely all NaN or no valid date differences for 'age_at_note'.")

    # Calculate AGE_AT_ADMISSION (more robustly with row-wise fallback for overflow)
    if 'dob' in df.columns and 'admittime' in df.columns:
        print("Attempting to calculate 'age_at_admission'.")

        # Initial filter for plausible dates
        plausible_date_mask = (
            df['dob'].notna() &
            df['admittime'].notna() &
            (df['admittime'] > df['dob']) &
            (df['dob'].dt.year >= 1850) # Filter out extremely early DOBs more strictly
            # Add a similar filter for admittime if you notice issues, e.g.
            # & (df['admittime'].dt.year < 2200)
        )

        print(f"Found {plausible_date_mask.sum()} rows with plausible DOB/ADMITTIME for age calculation.")

        # Define a function to calculate age for a single row, with overflow handling
        def calculate_age_row(row, dob_col='dob', event_date_col='admittime'):
            try:
                # Ensure dates are valid within this specific row context again
                if pd.notna(row[dob_col]) and pd.notna(row[event_date_col]) and row[event_date_col] > row[dob_col]:
                    # Check year again for safety, although primary filter should catch most
                    if row[dob_col].year < 1850: # or some other reasonable lower bound
                        return np.nan
                    age = (row[event_date_col] - row[dob_col]).days / 365.25
                    return age
                return np.nan
            except OverflowError:
                # This specific row caused an overflow, return NaN or a placeholder like -1
                # print(f"Overflow for subject_id {row.get('subject_id', 'Unknown')} with DOB {row[dob_col]} and ADMITTIME {row[event_date_col]}")
                return np.nan # Indicates an issue for this specific row
            except Exception: # Catch any other unexpected errors for a row
                return np.nan

        if plausible_date_mask.any():
            # Apply the row-wise calculation only to the plausible subset
            # This is slower than vectorized but necessary if OverflowError persists for some
            df.loc[plausible_date_mask, 'age_at_admission'] = df[plausible_date_mask].apply(
                calculate_age_row, axis=1, dob_col='dob', event_date_col='admittime'
            )

            # Age Capping for >89 y.o.
            # (patients de-identified as ~300 y.o. might still result in very large calculated ages before capping)
            # It's crucial to apply this after any age calculation method.
            df.loc[df['age_at_admission'] > 89, 'age_at_admission'] = 90.0

            # Report stats on successfully calculated ages
            calculated_ages_count = df['age_at_admission'].notna().sum()
            print(f"Calculated 'age_at_admission' for {calculated_ages_count} rows after row-wise processing and capping.")
            if calculated_ages_count > 0:
                 print(f"  Min age: {df['age_at_admission'].min():.2f}, Max age: {df['age_at_admission'].max():.2f}, Mean age: {df['age_at_admission'].mean():.2f}")
            if plausible_date_mask.sum() > calculated_ages_count:
                print(f"  Note: {plausible_date_mask.sum() - calculated_ages_count} rows within the 'plausible range' still resulted in NaN age (likely due to overflow during row-wise calculation or failed date conditions within calculate_age_row).")

        else:
            print("No rows found with plausible DOB/ADMITTIME after initial filtering for age calculation.")

        # Check for rows that didn't even make it into the plausible_date_mask
        if (~plausible_date_mask & df['dob'].notna() & df['admittime'].notna()).any():
            print(f"Note: Some additional rows had 'admittime' <= 'dob' or highly suspect DOB years (e.g., <1850); age was not attempted for these.")

    else:
        print("Required columns for age calculation ('dob' and 'admittime') not found.")

    # --- Clean ETHNICITY ---
    if 'ethnicity' in df.columns:
        df['ethnicity_clean'] = df['ethnicity'].fillna('UNKNOWN/NOT SPECIFIED')
        # print("Unique raw ethnicity values:", df['ethnicity'].unique()) # For tailoring map
        ethnicity_map = {
            # Prioritize more specific entries if they exist in your demo, then broader ones
            'WHITE': 'WHITE', 'WHITE - RUSSIAN': 'WHITE', 'WHITE - OTHER EUROPEAN': 'WHITE', 'WHITE - BRAZILIAN': 'WHITE',
            'BLACK/AFRICAN AMERICAN': 'BLACK', 'BLACK/CAPE VERDEAN': 'BLACK', 'BLACK/HAITIAN': 'BLACK', 'BLACK/AFRICAN': 'BLACK',
            'HISPANIC OR LATINO': 'HISPANIC/LATINO', 'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
            'HISPANIC/LATINO - DOMINICAN': 'HISPANIC/LATINO', 'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
            'ASIAN': 'ASIAN', 'ASIAN - CHINESE': 'ASIAN', 'ASIAN - ASIAN INDIAN': 'ASIAN', 'ASIAN - VIETNAMESE': 'ASIAN',
            'ASIAN - FILIPINO': 'ASIAN', 'ASIAN - CAMBODIAN': 'ASIAN', 'ASIAN - KOREAN': 'ASIAN',
            'UNKNOWN/NOT SPECIFIED': 'UNKNOWN/NOT SPECIFIED',
            'UNABLE TO OBTAIN': 'UNKNOWN/NOT SPECIFIED',
            'PATIENT DECLINED TO ANSWER': 'UNKNOWN/NOT SPECIFIED',
            'AMERICAN INDIAN/ALASKA NATIVE': 'OTHER', # Or specific category if desired and N is sufficient
            'MULTI RACE ETHNICITY': 'OTHER',
            'PORTUGUESE': 'WHITE',
            'MIDDLE EASTERN': 'OTHER',
            'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'OTHER',
            'CARIBBEAN ISLAND': 'OTHER',
            'SOUTH AMERICAN': 'HISPANIC/LATINO', # Check if this is appropriate for your demo's values
            'OTHER': 'OTHER', # Catch-all for unmapped non-NaNs
        }
        # Ensure correct application of map, converting input to str and upper for map keys
        df['ethnicity_clean'] = df['ethnicity'].astype(str).str.strip().str.upper().map(ethnicity_map).fillna(
            df['ethnicity'].apply(lambda x: 'UNKNOWN/NOT SPECIFIED' if pd.isna(x) or str(x).strip()=='' else 'OTHER')
        )
        print("\nCleaned Ethnicity Counts:")
        print(df['ethnicity_clean'].value_counts(dropna=False))
    else:
        print("Column 'ethnicity' not found in DataFrame.")

    # --- Clean LANGUAGE ---
    if 'language' in df.columns:
        def map_language_simple(lang):
            if pd.isna(lang): return 'MISSING'
            lang_str = str(lang).strip().upper()
            if lang_str in ['ENGL', 'ENGLISH', 'EN', 'ENG']: return 'ENGLISH'
            if lang_str in ['?', '', 'UNABLE TO OBTAIN']: return 'MISSING'
            return 'NON-ENGLISH'
        df['language_clean'] = df['language'].apply(map_language_simple)
        print("\nCleaned Language Counts:")
        print(df['language_clean'].value_counts(dropna=False))
    else:
        print("Column 'language' not found in DataFrame.")

    print("\nDataFrame sample after cleaning and feature engineering:")
    cols_to_show = ['subject_id', 'hadm_id', 'ethnicity_clean', 'language_clean']
    if 'row_id_y' in df.columns: cols_to_show.insert(2, 'row_id_y')
    if 'age_at_admission' in df.columns: cols_to_show.append('age_at_admission')
    if 'age_at_note' in df.columns: cols_to_show.append('age_at_note')
    if 'icd9_codes_list' in df.columns: cols_to_show.append('icd9_codes_list')
    if 'category_note' in df.columns: cols_to_show.append('category_note')
    final_cols_to_show = [col for col in cols_to_show if col in df.columns]
    print(df[final_cols_to_show].head(3).to_string())
else:
    print("df is None. Skipping cleaning and feature engineering.")

OverflowError encountered during 'age_at_note' calculation for some rows. These will remain NaN.
Attempting to calculate 'age_at_admission'.
Found 1823440 rows with plausible DOB/ADMITTIME for age calculation.
Calculated 'age_at_admission' for 1790894 rows after row-wise processing and capping.
  Min age: 0.00, Max age: 89.00, Mean age: 48.03
  Note: 32546 rows within the 'plausible range' still resulted in NaN age (likely due to overflow during row-wise calculation or failed date conditions within calculate_age_row).
Note: Some additional rows had 'admittime' <= 'dob' or highly suspect DOB years (e.g., <1850); age was not attempted for these.

Cleaned Ethnicity Counts:
ethnicity_clean
WHITE                    1298765
UNKNOWN/NOT SPECIFIED     192465
BLACK                     180462
HISPANIC/LATINO            63150
OTHER                      62845
ASIAN                      54272
Name: count, dtype: int64

Cleaned Language Counts:
language_clean
MISSING        877738
ENGLISH        827

## **Block 5: Selecting Relevant Data & Saving Output**

> In this final preprocessing step, we:
>
> * Choose only the columns most useful for bias-analysis tasks (e.g., template generation or simple downstream predictions).
> * Save the resulting DataFrame as a pickle file to enable fast reloading in your main analysis notebook—no need to rerun all preprocessing each time.
>
> **Reminder:**
>
> * Inspect and adjust your `keep_cols` list to include exactly the fields your downstream tasks require.
> * Confirm your output path and filename before saving.

---

In [ ]:
# Block 5: Selecting Relevant Data and Saving Output

if df is not None:
    # Define columns relevant for your analysis.
    # All column names should be lowercase as per your previous standardization.
    # These should match the columns present in your 'df' after Block 4.
    columns_to_keep = [
        'subject_id', 'hadm_id',
        'row_id_x', 'row_id_y', # From original patient/admission tables

        # Note details (these are expected to be mostly NaN if NOTEEVENTS.csv was from demo/empty)
        'category_note', 'text_note', 'chartdate_note', 'charttime_note', 'row_id_note',

        # Patient demographics & admission details
        'gender', 'dob', # dob is used for age calculation, might not be needed in final processed_df
        'admittime', 'dischtime', 'deathtime', 'dod',
        'dod_hosp', 'dod_ssn', 'expire_flag', # expire_flag from patients, hospital_expire_flag from admissions
        'insurance', 'language_clean', 'ethnicity_clean',
        'religion', 'marital_status', 'admission_type', 'admission_location',
        'discharge_location', 'diagnosis', # Admission diagnosis
        'hospital_expire_flag',

        # Engineered features
        'age_at_admission', 'age_at_note', # age_at_note will be mostly NaN
        'icd9_codes_list'
    ]

    # Filter this list to only include columns that actually exist in your current 'df'
    columns_to_keep_existing = [col for col in columns_to_keep if col in df.columns]
    print(f"Attempting to select these existing columns: {columns_to_keep_existing}")

    processed_df = df[columns_to_keep_existing].copy()
    print(f"Shape of processed_df after selecting columns: {processed_df.shape}")

    # Optional: Further filtering, e.g., by note category, could be done here if 'category_note' had data.
    # Since it's likely NaN, this step would typically be skipped or be ineffective.
    # Example:
    # if 'category_note' in processed_df.columns and processed_df['category_note'].notna().any():
    #     desired_categories = ['Discharge summary'] # Example
    #     processed_df = processed_df[processed_df['category_note'].isin(desired_categories)]
    #     print(f"Shape after filtering by note category (if applicable): {processed_df.shape}")

    # --- Critical dropna() step ---
    # Drop rows where *truly essential* columns for your planned analysis are missing.
    # Based on your previous successful run, these columns were key.
    essential_cols_for_analysis = ['subject_id', 'hadm_id', 'gender', 'ethnicity_clean', 'age_at_admission']

    # Ensure these essential columns actually exist in processed_df before using them in subset
    existing_essential_cols = [col for col in essential_cols_for_analysis if col in processed_df.columns]

    if existing_essential_cols:
        print(f"Dropping rows where any of these essential columns are NaN: {existing_essential_cols}")
        # how='any' means drop the row if any of the specified columns in 'subset' is NaN
        processed_df.dropna(subset=existing_essential_cols, how='any', inplace=True)
        print(f"Shape of processed DataFrame after dropping NaNs in essential columns: {processed_df.shape}")
    else:
        print("Warning: One or more essential columns for dropna were not found. Skipping this dropna step.")

    if processed_df.empty:
        print("WARNING: processed_df is empty after dropna(). This means all rows had NaNs in one of the essential columns.")
        print("Check your data and the 'essential_cols_for_analysis' list.")
    else:
        print(f"Final shape of processed DataFrame before saving: {processed_df.shape}")

        # --- Define output path ---
        OUTPUT_PROCESSED_DIR = "/content/drive/MyDrive/CS598 Final Project/" # Ensure this GDrive path exists

        # Create the directory if it doesn't exist (for Google Drive path)
        if OUTPUT_PROCESSED_DIR.startswith("/content/drive/") and not os.path.exists(OUTPUT_PROCESSED_DIR):
            try:
                os.makedirs(OUTPUT_PROCESSED_DIR)
                print(f"Created directory for output: {OUTPUT_PROCESSED_DIR}")
            except Exception as e:
                print(f"Could not create directory {OUTPUT_PROCESSED_DIR}. Error: {e}")
                # Fallback to local Colab storage if Drive directory creation fails
                print("Falling back to saving in Colab local storage './'")
                OUTPUT_PROCESSED_DIR = "./"


        OUTPUT_PROCESSED_PATH_PKL = os.path.join(OUTPUT_PROCESSED_DIR, "processed_mimic_data.pkl") # Renamed slightly
        OUTPUT_PROCESSED_PATH_CSV = os.path.join(OUTPUT_PROCESSED_DIR, "processed_mimic_data.csv") # Renamed slightly

        try:
            processed_df.to_pickle(OUTPUT_PROCESSED_PATH_PKL)
            print(f"Processed data saved to pickle: {OUTPUT_PROCESSED_PATH_PKL}")
            processed_df.to_csv(OUTPUT_PROCESSED_PATH_CSV, index=False)
            print(f"Processed data saved to CSV: {OUTPUT_PROCESSED_PATH_CSV}")

            print("\nFinal Processed DataFrame sample:")
            # Displaying a sample of the final processed data
            # Use .to_string() if you want to see more of the wide DataFrame without truncation
            print(processed_df.head(3).to_string())

        except Exception as e:
            print(f"Error saving processed data: {e}")
            if OUTPUT_PROCESSED_DIR.startswith("/content/drive/"):
                 print("Ensure Google Drive is correctly mounted and you have write permissions to the specified path.")

else:
    print("df is None (was not created in previous steps). Skipping final selection and saving.")

Attempting to select these existing columns: ['subject_id', 'hadm_id', 'row_id_x', 'row_id_y', 'category_note', 'text_note', 'chartdate_note', 'charttime_note', 'row_id_note', 'gender', 'dob', 'admittime', 'dischtime', 'deathtime', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'insurance', 'language_clean', 'ethnicity_clean', 'religion', 'marital_status', 'admission_type', 'admission_location', 'discharge_location', 'diagnosis', 'hospital_expire_flag', 'age_at_admission', 'age_at_note', 'icd9_codes_list']
Shape of processed_df after selecting columns: (1851959, 31)
Dropping rows where any of these essential columns are NaN: ['subject_id', 'hadm_id', 'gender', 'ethnicity_clean', 'age_at_admission']
Shape of processed DataFrame after dropping NaNs in essential columns: (1790894, 31)
Final shape of processed DataFrame before saving: (1790894, 31)
Processed data saved to pickle: /content/drive/MyDrive/CS598 Final Project/processed_mimic_data.pkl
Processed data saved to CSV: /content/drive/M

In [ ]:
# Block 5: Selecting Relevant Data and Saving Output

if df is not None:
    # Define columns relevant for your analysis.
    # These should match the lowercase column names in your 'df' after Block 4.
    columns_to_keep = [
        'subject_id', 'hadm_id',
        'row_id_x', 'row_id_y', # From original patient/admission tables. Consider if both are needed or choose one.

        # Note details - these will now have data for many rows
        'category_note', 'text_note', 'chartdate_note', # charttime_note, row_id_note might be less critical

        # Patient demographics & admission details
        'gender',
        'admittime', # dob, dischtime, deathtime, dod, etc., might be kept if directly needed for analysis, else age is derived
        'expire_flag', # from patients table
        'insurance', 'language_clean', 'ethnicity_clean',
        'diagnosis', # Admission diagnosis
        'hospital_expire_flag', # from admissions table

        # Engineered features
        'age_at_admission', # This is your most reliable age column now
        # 'age_at_note', # Keep if populated and useful, otherwise might be mostly NaN
        'icd9_codes_list'
    ]

    # Filter this list to only include columns that actually exist in your current 'df'
    columns_to_keep_existing = [col for col in columns_to_keep if col in df.columns]
    print(f"Attempting to select these existing columns: {columns_to_keep_existing}")

    processed_df = df[columns_to_keep_existing].copy()
    print(f"Shape of processed_df after selecting columns: {processed_df.shape}")

    # --- Crucial dropna() step ---
    # Now that 'text_note' has data, you might want to ensure it's present for text-based analyses.
    # Also ensure key demographics and your primary age column are present.
    essential_cols_for_analysis = [
        'subject_id', 'hadm_id', 'gender',
        'ethnicity_clean', 'age_at_admission',
        'text_note', # Now important if downstream tasks use text
        'hospital_expire_flag' # If this is your primary label for a downstream task
    ]

    # Ensure these essential columns actually exist in processed_df
    existing_essential_cols = [col for col in essential_cols_for_analysis if col in processed_df.columns]

    if existing_essential_cols:
        print(f"Dropping rows where any of these essential columns are NaN: {existing_essential_cols}")
        processed_df.dropna(subset=existing_essential_cols, how='any', inplace=True)
        print(f"Shape of processed DataFrame after dropping NaNs in essential columns: {processed_df.shape}")
    else:
        print("Warning: One or more essential columns for dropna were not found. Skipping this dropna step.")

    if processed_df.empty:
        print("WARNING: processed_df is empty after dropna(). This means all rows had NaNs in one of the essential columns.")
        print("Check your data and the 'essential_cols_for_analysis' list. Consider if 'text_note' being NaN is acceptable for some rows if not all notes are critical.")
    else:
        print(f"Final shape of processed DataFrame before saving: {processed_df.shape}")

        # --- Define output path ---
        OUTPUT_PROCESSED_DIR = "/content/drive/MyDrive/CS598 Final Project/" # Your Google Drive path

        if OUTPUT_PROCESSED_DIR.startswith("/content/drive/") and not os.path.exists(OUTPUT_PROCESSED_DIR):
            try:
                os.makedirs(OUTPUT_PROCESSED_DIR)
                print(f"Created directory for output: {OUTPUT_PROCESSED_DIR}")
            except Exception as e:
                print(f"Could not create directory {OUTPUT_PROCESSED_DIR}. Error: {e}")
                print("Falling back to saving in Colab local storage './'")
                OUTPUT_PROCESSED_DIR = "./"

        # Use a new name to distinguish from demo-based processed file
        OUTPUT_PROCESSED_PATH_PKL = os.path.join(OUTPUT_PROCESSED_DIR, "processed_mimic_full_subset.pkl")
        OUTPUT_PROCESSED_PATH_CSV = os.path.join(OUTPUT_PROCESSED_DIR, "processed_mimic_full_subset.csv")

        try:
            print(f"Attempting to save processed data ({len(processed_df)} rows) to {OUTPUT_PROCESSED_DIR}...")
            # Saving large DataFrames, especially to Drive, can take time.
            processed_df.to_pickle(OUTPUT_PROCESSED_PATH_PKL)
            print(f"Processed data saved to pickle: {OUTPUT_PROCESSED_PATH_PKL}")
            # Saving to CSV can be very slow for large text data and large files. Consider if essential.
            # If you save to CSV, be prepared for it to take a while and create a large file.
            # For faster inspection, you might save a sample: processed_df.head(1000).to_csv(...)
            # For now, let's save the full one.
            processed_df.to_csv(OUTPUT_PROCESSED_PATH_CSV, index=False)
            print(f"Processed data saved to CSV: {OUTPUT_PROCESSED_PATH_CSV}")

            print("\nFinal Processed DataFrame sample (first 3 rows):")
            print(processed_df.head(3).to_string())

        except Exception as e:
            print(f"Error saving processed data: {e}")
            if OUTPUT_PROCESSED_DIR.startswith("/content/drive/"):
                 print("Ensure Google Drive is correctly mounted and you have write permissions.")
else:
    print("df is None (was not created in previous steps). Skipping final selection and saving.")

Attempting to select these existing columns: ['subject_id', 'hadm_id', 'row_id_x', 'row_id_y', 'category_note', 'text_note', 'chartdate_note', 'gender', 'admittime', 'expire_flag', 'insurance', 'language_clean', 'ethnicity_clean', 'diagnosis', 'hospital_expire_flag', 'age_at_admission', 'icd9_codes_list']
Shape of processed_df after selecting columns: (1851959, 17)
Dropping rows where any of these essential columns are NaN: ['subject_id', 'hadm_id', 'gender', 'ethnicity_clean', 'age_at_admission', 'text_note', 'hospital_expire_flag']
Shape of processed DataFrame after dropping NaNs in essential columns: (1790301, 17)
Final shape of processed DataFrame before saving: (1790301, 17)
Attempting to save processed data (1790301 rows) to /content/drive/MyDrive/CS598 Final Project/...
Processed data saved to pickle: /content/drive/MyDrive/CS598 Final Project/processed_mimic_full_subset.pkl
Processed data saved to CSV: /content/drive/MyDrive/CS598 Final Project/processed_mimic_full_subset.csv





## Next Steps & Reminders

This structure gives you a solid starting point for your `data_preprocessing.ipynb`. Keep in mind:

* **Adjust CSV filenames & paths** in Block 2 to match where your demo files are stored.
* **Inspect unique values** for `ETHNICITY` and `LANGUAGE` after Block 3, then refine your mapping logic in Block 4.
* **Select only the essential columns** for your downstream bias-analysis in the final DataFrame.
* **Review the age calculation**—choose the most reliable date fields in your demo CSVs to get the best estimates.

> By following this simplified pipeline, you’ll end up with a lightweight, reusable dataset tailored for your core bias-analysis tasks.